In [1]:
import transtab

# load multiple datasets by passing a list of data names
allset, trainset, valset, testset, cat_cols, num_cols, bin_cols \
    = transtab.load_data(['.\\data\\dataset1','.\\data\\dataset2', '.\\data\\dataset3'])
# allset, trainset, valset, testset, cat_cols, num_cols, bin_cols = transtab.load_data(['credit-g'])
# 预测目标需要自己指定列名为target_label


h:\Anaconda3\envs\transtab\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


########################################
load from local data dir .\data\dataset1
# data: 1189, # feat: 10, # cate: 0,  # bin: 0, # numerical: 10, pos rate: 0.71
########################################
load from local data dir .\data\dataset2
# data: 118201, # feat: 50, # cate: 0,  # bin: 9, # numerical: 41, pos rate: 0.49
########################################
load from local data dir .\data\dataset3
# data: 726, # feat: 22, # cate: 0,  # bin: 0, # numerical: 22, pos rate: 0.74


In [4]:
valset

[(           age     heart     liver    spleen      lung    kidney     brain   
  532   0.401575  0.456140  0.576376  0.375788  0.355465  0.476534  0.737416  \
  720   0.517060  0.628655  0.676799  0.171501  0.650407  0.633574  0.801245   
  348   0.307087  0.336257  0.394767  0.137453  0.703252  0.389892  0.685522   
  671   0.496063  0.732456  0.757214  0.186633  0.671183  0.788809  0.765957   
  463   0.275591  0.241228  0.499808  0.039092  0.293135  0.341155  0.634146   
  ...        ...       ...       ...       ...       ...       ...       ...   
  545   0.307087  0.679825  0.802616  0.163934  0.549232  0.882671  0.652309   
  1087  0.338583  0.467836  0.461716  0.127364  0.442638  0.444043  0.650752   
  573   0.905512  0.450292  0.279723  0.080706  0.272809  0.350181  0.582252   
  413   0.464567  0.502924  0.760677  0.279950  0.396116  0.718412  0.666321   
  315   0.296588  0.510234  0.636783  0.264817  0.341012  0.635379  0.656980   
  
        pancreas    height        id 

In [ ]:
# build transtab classifier model
# model = transtab.build_classifier(cat_cols, num_cols, bin_cols)
model, collate_fn = transtab.build_contrastive_learner(
    cat_cols, num_cols, bin_cols,
    supervised=False,
    num_partition=4,
    overlap_ratio=0.5
)

# specify training arguments, take validation loss for early stopping
training_arguments = {
    'num_epoch': 50,
    'batch_size':64,
    'lr':1e-4,
    'eval_metric': 'val_loss',
    'eval_less_is_better': True,
    'output_dir': './checkpoint'
}

transtab.train(model, trainset, valset[0],collate_fn=collate_fn, **training_arguments)

In [ ]:
# load the pretrained model and finetune on a target dataset
allset, trainset, valset, testset, cat_cols, num_cols, bin_cols \
    = transtab.load_data('credit-approval')

# build transtab classifier model, and load from the pretrained dir
model = transtab.build_classifier(checkpoint='./checkpoint')

# update model's categorical/numerical/binary column dict
model.update({'cat': cat_cols, 'num': num_cols, 'bin': bin_cols})

# 监督学习
# ypred = transtab.predict(model, x_test) # 注意：预测分类标签返回值在[0,1]之间，需要手动修改
# print(transtab.evaluate(ypred, y_test, seed=123, metric='auc')) # 评估